In [1]:
import SimpleITK as sitk
import sys
import os
import cv2

data_directory = 'C:/Users/Gurkan/Desktop/code2/1_08_P/'
count=8
mr_ct_count = 1
img_reg_count = 1

output_file = 'C:/Users/oguzt/Desktop/gürkan.a/p'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed
from IPython.display import clear_output

# Callback invoked by the interact IPython method for scrolling through the image stacks of
# the two images (moving and fixed).
def display_images(fixed_image_z, moving_image_z, fixed_npa, moving_npa):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1,2,figsize=(10,8))
    
    # Draw the fixed image in the first subplot.
    plt.subplot(1,2,1)
#    plt.imshow(fixed_npa[fixed_image_z,:,:],cmap=plt.cm.Greys_r);
    plt.title('fixed image')
#    plt.imsave(outputImageFileNameCt +str(mr_ct_count)+'.jpeg',fixed_npa[fixed_image_z,:,:] ,cmap="gray")
    plt.axis('off')
    
#    writer = sitk.ImageFileWriter()
#    writer.SetFileName(outputImageFileNameCt)
#    writer.Execute(image)
    
    
    # Draw the moving image in the second subplot.
    plt.subplot(1,2,2)
#    plt.imshow(moving_npa[moving_image_z,:,:],cmap=plt.cm.Greys_r);
#    plt.imsave(outputImageFileNameMr +str(mr_ct_count)+'.jpeg',moving_npa[moving_image_z,:,:] ,cmap="gray")
    plt.title('moving image')
    plt.axis('off')
    
    
#    writer = sitk.ImageFileWriter()
#    writer.SetFileName(outputImageFileNameMr)
#    writer.Execute(image)
#    mr_ct_count += 1
    plt.show()

# Callback invoked by the IPython interact method for scrolling and modifying the alpha blending
# of an image stack of two images that occupy the same physical space. 
def display_images_with_alpha(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[:,:,image_z] + alpha*moving[:,:,image_z] 
    plt.imshow(sitk.GetArrayViewFromImage(img),cmap=plt.cm.Greys_r);
    plt.axis('off')
#    plt.show()
    
#    import cv2
#    cv2.imwrite(outputImageFileNameResult +str(img_reg_count)+'.tif', sitk.GetArrayViewFromImage(img))
    
   # img_reg_count++
    
    
# Callback invoked when the StartEvent happens, sets up our new data.
def start_plot():
    global metric_values, multires_iterations
    
    metric_values = []
    multires_iterations = []

# Callback invoked when the EndEvent happens, do cleanup of data and figure.
def end_plot():
    global metric_values, multires_iterations
    
    del metric_values
    del multires_iterations
    # Close figure, we don't want to get a duplicate of the plot latter on.
    plt.close()

# Callback invoked when the IterationEvent happens, update our data and display new figure.
def plot_values(registration_method):
    global metric_values, multires_iterations
    
    metric_values.append(registration_method.GetMetricValue())                                       
    # Clear the output area (wait=True, to reduce flickering), and plot current data
    clear_output(wait=True)
    # Plot the similarity metric values
    plt.plot(metric_values, 'r')
    plt.plot(multires_iterations, [metric_values[index] for index in multires_iterations], 'b*')
    plt.xlabel('Iteration Number',fontsize=12)
    plt.ylabel('Metric Value',fontsize=12)
    plt.show()
    
# Callback invoked when the sitkMultiResolutionIterationEvent happens, update the index into the 
# metric_values list. 
def update_multires_iterations():
    global metric_values, multires_iterations
    multires_iterations.append(len(metric_values))

In [3]:

patient_numbers = 19

for pnum in range(19, patient_numbers + 1):
    fixed_image =  sitk.ReadImage('C:/Users/oguzt/Desktop/Datasets/p'+ str(pnum) +'/ct/ct.mhd')
    moving_image = sitk.ReadImage('C:/Users/oguzt/Desktop/Datasets/p'+ str(pnum) +'/mr_T1/mr.mhd') 

    initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                          moving_image, 
                                                          sitk.Euler3DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)

    moving_resampled = sitk.Resample(moving_image, fixed_image, initial_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings.
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)

    registration_method.SetInterpolator(sitk.sitkLinear)

    # Optimizer settings.
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework.            
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Don't optimize in-place, we would possibly like to run this cell multiple times.
    registration_method.SetInitialTransform(initial_transform, inPlace=False)

    # Connect all of the observers so that we can perform plotting during registration.
    registration_method.AddCommand(sitk.sitkStartEvent, start_plot)
    registration_method.AddCommand(sitk.sitkEndEvent, end_plot)
    registration_method.AddCommand(sitk.sitkMultiResolutionIterationEvent, update_multires_iterations) 
    registration_method.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration_method))

    final_transform = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                                   sitk.Cast(moving_image, sitk.sitkFloat32))

    moving_resampled = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

    path = output_file + str(pnum) + '/'
    os.mkdir(path)
    
    
    path_i = path + 'ct/'
    os.mkdir(path_i)
    ct_scans = sitk.GetArrayFromImage(fixed_image)
    for i in range(ct_scans.shape[0]):
        plt.imsave(path_i +str(i+1)+'.jpeg', ct_scans[i] ,cmap="gray")
    #plt.show()

    path_i = path + 'mr/'
    os.mkdir(path_i)
    mr_scans = sitk.GetArrayFromImage(moving_image)
    for i in range(mr_scans.shape[0]):
        plt.imsave(path_i +str(i+1)+'.jpeg', mr_scans[i] ,cmap="gray")
    #plt.show()
    
    path_i = path + 'result/'
    os.mkdir(path_i)
    results = sitk.GetArrayFromImage(moving_resampled)
    for i in range(results.shape[0]):
        #cv2.imwrite(path_i +str(count)+'.tif', results[i])
        plt.imsave(path_i +str(i+1)+'.tiff', results[i] ,cmap="gray")
    #plt.show()




RuntimeError: Exception thrown in SimpleITK ImageRegistrationMethod_Execute: d:\a\1\work\b\itk-prefix\include\itk-5.1\itkImageBase.hxx:177:
itk::ERROR: itk::ERROR: Image(000001E9EE62ECB0): A spacing of 0 is not allowed: Spacing is [0, 0]